In [ ]:
import bw2data as bd
import bw2io as bi
import bw2regional as bwr

We use the project which already has ecoinvent as a background.

In [ ]:
bd.projects.set_current("LC IMPACT case study")

In [ ]:
if 'meal' in bd.databases:
    del bd.databases['meal']

In [ ]:
ei = bd.Database('ecoinvent')
bio = bd.Database('biosphere3')

Find the background nodes in ecoinvent.

In [ ]:
wheat = ei.get(name='wheat production', location='ES')

In [ ]:
water = bio.get(
    name='Water', 
    categories=('water', 'surface water')
)

In [ ]:
elec = ei.get(
    name='market for electricity, low voltage', 
    location='ES'
)

When searching for things, it can be convenient to use a list comprehension to filter by lower case, etc.

In [ ]:
[node 
     for node in ei 
     if node['location'] == 'ES' 
     and 'low voltage' in node['name']
     and 'market' in node['name']
]

We can look over the biosphere edges to find the water we want (though this is already fixed above).

In [ ]:
for edge in wheat.biosphere():
    if (
        edge.input['name'] == 'Water' 
        and 'surface' in str(edge.input.get('categories'))
    ):
        print(edge)
        break

Define the geocollections for our spatial units.

In [ ]:
bwr.geocollections['BCN'] = {
    'filepath': '/srv/data/vectors/barc_polygon.geojson',
    'field': 'identifier',
}

In [ ]:
bwr.geocollections['ALM'] = {
    'filepath': '/srv/data/vectors/alm2.geojson',
    'field': 'id',
}

Geocollections. Do they know things? Let's find out!

In [ ]:
bwr.geocollections['ALM']

Finally we get to the nodes and edges...

In [ ]:
db = bd.Database('meal')
db.register()

In [ ]:
meal = db.new_node(
    code='meal',
    name='really good meal trust me',
    unit='mecal',
    location=('BCN', 'ILCA'),
    ilca_course_barca=True,
)
meal.save()

In [ ]:
meal.new_edge(
    input=elec,
    amount=2,
    type='technosphere',
).save()

In [ ]:
meal.new_edge(
    input=wheat,
    amount=0.5,
    type='technosphere',
).save()

In [ ]:
meal.new_edge(
    input=water,
    amount=1,
    type='biosphere',
).save()

In [ ]:
meal.new_edge(
    input=meal,
    amount=1,
    type='production',
).save()

In [ ]:
tomato = db.new_node(
    code='🍅',
    name='🍅',
    unit='kg',
    location=('ALM', '1'),
)
tomato.save()

In [ ]:
meal.new_edge(
    input=tomato,
    amount=0.25,
    type='technosphere',
).save()

In [ ]:
tomato.new_edge(
    input=elec,
    amount=1.5,
    type='technosphere',
).save()

In [ ]:
tomato.new_edge(
    input=water,
    amount=5,
    type='biosphere',
).save()

In [ ]:
for node in db:
    print(node, node['code'])
    for edge in node.edges():
        print("\t\t", edge)

Run if needed...

In [ ]:
meal.delete_duplicate_exchanges()